<h1 style="color: #00BFFF;">00 |</h1>

In [1]:
# 📚 Basic libraries
import os # file managment
import matplotlib.pyplot as plt #  plots and visualizations
import numpy as np # numerical python, image array manipulation

# 🛠️ Tools
import warnings # who likes warnings?
import shutil # high-level file operations
import random # to generate random samples

# 🌐 Computer Vision
import cv2 # the most basic library to read images
from tensorflow.keras.preprocessing.image import ImageDataGenerator # real-time data augmentation
from tensorflow.keras.utils import img_to_array, array_to_img, load_img # saving augmented Data
from tensorflow.keras import layers, Model
from tensorflow import keras # deep learning and neural networks
from keras.models import load_model

In [2]:
# ⚙️ Settings
warnings.filterwarnings('ignore') # ignore warnings

In [3]:
# 🎯

<h1 style="color: #00BFFF;">01 | Data Extraction</h1>

In [4]:
data_path = os.path.join('C:\\Users\\apisi\\01. IronData\\01. GitHub\\03. Projects\\07_hands_on_it', '01_data')

<h3 style="color: #008080;">Preparing Test folder</h3>

In [5]:
test_dir = os.path.join(data_path, "03_test")
os.makedirs(test_dir, exist_ok=True)
# original and no_bg directories for test
os.makedirs(os.path.join(test_dir, "original"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "mask"), exist_ok=True)

<h3 style="color: #008080;">Preparing Train and Validation folder</h3>

In [6]:
train_dir = os.path.join(data_path, "01_train")
val_dir = os.path.join(data_path, "02_validation")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
# original and no_bg directories for train and validation
os.makedirs(os.path.join(train_dir, "original"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "mask"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "original"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "mask"), exist_ok=True)

<h1 style="color: #00BFFF;">02 | Modeling</h1>

<h3 style="color: #00BFFF;">Improving is an iterative process...</h3>

<h3 style="color: #008080;">Preparing Test folder</h3>

In [7]:
test_dir = os.path.join(data_path, "03_test")
os.makedirs(test_dir, exist_ok=True)
# original and no_bg directories for test
os.makedirs(os.path.join(test_dir, "original"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "mask"), exist_ok=True)

<h3 style="color: #008080;">Preparing Train and Validation folder</h3>

In [8]:
train_dir = os.path.join(data_path, "01_train")
val_dir = os.path.join(data_path, "02_validation")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
# original and no_bg directories for train and validation
os.makedirs(os.path.join(train_dir, "original"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "mask"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "original"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "mask"), exist_ok=True)

In [9]:
# Parameters that we can fine-tune later on
img_height = 480
img_width = 272
image_size = (img_height, img_width)
batch_size = 16

<h3 style="color: #008080;">Normalization</h3>

In [10]:
train_image_datagen = ImageDataGenerator(rescale=1./255)
train_mask_datagen = ImageDataGenerator(rescale=1./255)

val_image_datagen = ImageDataGenerator(rescale=1./255)
val_mask_datagen = ImageDataGenerator(rescale=1./255)

<h3 style="color: #008080;">Debugging ImageDataGenerator</h3>

(wasn't detecting the images, they need to be in subfolders)

In [ ]:
train_original_dir = os.path.join(train_dir, "original")
train_mask_dir = os.path.join(train_dir, "mask")

val_original_dir = os.path.join(val_dir, "original")
val_mask_dir = os.path.join(val_dir, "mask")

<h3 style="color: #008080;">Train and Validation Data</h3>

In [12]:
train_image_gen = train_image_datagen.flow_from_directory(
    train_original_dir,
    class_mode=None,
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=False
)

train_mask_gen = train_mask_datagen.flow_from_directory(
    train_mask_dir,
    class_mode=None,
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=False
)

Found 249 images belonging to 1 classes.
Found 249 images belonging to 1 classes.


In [13]:
train_ds = zip(train_image_gen, train_mask_gen)

In [14]:
val_image_gen = val_image_datagen.flow_from_directory(
    val_original_dir,
    class_mode=None,
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=False
)

val_mask_gen = val_mask_datagen.flow_from_directory(
    val_mask_dir,
    class_mode=None,
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=False
)

Found 7 images belonging to 1 classes.
Found 7 images belonging to 1 classes.


In [15]:
val_ds = zip(val_image_gen, val_mask_gen)

<h3 style="color: #008080;">Compiling the Model</h3>

In [16]:
model = unet_resnet_model(input_shape=(img_height, img_width, 3))

In [17]:
model.compile(
    optimizer=keras.optimizers.Adamax(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [18]:
steps_per_epoch = len(train_image_gen)
validation_steps = len(val_image_gen)

In [19]:
checkpoints = os.path.join(data_path, "04_epochs", "save_at_transfer_{epoch}.keras")

epochs = 30

callbacks = [
    keras.callbacks.ModelCheckpoint(checkpoints),
]

hist = model.fit(
    train_ds, 
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=callbacks
)

<h1 style="color: #00BFFF;">06 | Evaluating the Model</h1>

In [21]:
#  path to best epoch
checkpoint_path = os.path.join(data_path, "04_epochs", "save_at_transfer_30.keras")

# load the model
loaded_model = load_model(checkpoint_path)

In [22]:
test_original_dir = os.path.join(test_dir, "original")
test_mask_dir = os.path.join(test_dir, "mask")

In [ ]:
test_image_datagen = ImageDataGenerator()
test_mask_datagen = ImageDataGenerator()

test_image_gen = test_image_datagen.flow_from_directory(
    test_original_dir,
    class_mode=None,
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=False
)

test_mask_gen = test_mask_datagen.flow_from_directory(
    test_mask_dir,
    class_mode=None,
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=False
)

test_ds = zip(test_image_gen, test_mask_gen)

# Evaluate the model
loss, accuracy = loaded_model.evaluate(test_ds, steps=len(test_image_gen))

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

images, masks = next(test_ds)

# predict masks using the model
predicted_masks = loaded_model.predict(images)

# plotting original, true masks, predicted masks, and segmented images
for i in range(min(5, batch_size)):
    plt.figure(figsize=(15, 5))

    # Display original image
    plt.subplot(1, 4, 1)
    image_to_display = images[i].astype(np.uint8)
    plt.imshow(image_to_display)
    plt.title('Original Image')

    # Display true mask
    plt.subplot(1, 4, 2)
    plt.imshow(masks[i].squeeze(), cmap='gray')
    plt.title('True Mask')

    # Display predicted mask
    plt.subplot(1, 4, 3)
    plt.imshow(predicted_masks[i].squeeze(), cmap='gray')
    plt.title('Predicted Mask')
    
    # Display segmented image
    plt.subplot(1, 4, 4)
    segmented_image = (images[i] * predicted_masks[i].squeeze()[:, :, np.newaxis]).astype(np.uint8)
    plt.imshow(segmented_image)
    plt.title('Segmented Image')
    
    plt.tight_layout()
    plt.show()